### Projeto 3: operando no metatrader e comprando as acoes indicadas pelo modelo

passo 1 -  rodar aula anterior

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


In [11]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
time.sleep(3)

url = "https://www.fundamentus.com.br/resultado.php"
driver.get(url)

local_tabela = "/html/body/div[1]/div[2]/table"
elemento = driver.find_element("xpath", local_tabela)
html_tabela = elemento.get_attribute('outerHTML')
tabela = pd.read_html(str(html_tabela), thousands=".")[0]

tabela = tabela.set_index("Papel")
for col in tabela.columns:
    
    tabela[col] = tabela[col].str.replace("%", "")
    tabela[col] = tabela[col].str.replace(".", "")
    tabela[col] = tabela[col].str.replace(",", ".")
    tabela[col] = tabela[col].astype(float)

tabela = tabela[["Cotação", "EV/EBIT", "ROIC", "Liq.2meses"]]
tabela = tabela[tabela["Liq.2meses"] > 1000000]
tabela = tabela[tabela["EV/EBIT"] > 0]
tabela = tabela[tabela["ROIC"] > 0]

tabela["ranking_ev_ebit"] = tabela["EV/EBIT"].rank(ascending= True)
tabela["ranking_roic"] = tabela["ROIC"].rank(ascending= False)
tabela["ranking_final"] = tabela["ranking_ev_ebit"] + tabela["ranking_roic"]
tabela = tabela.sort_values("ranking_final")

driver.quit()


C:\Users\Risonaldo\AppData\Local\Temp\ipykernel_5668\2437173279.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabela = pd.read_html(str(html_tabela), thousands=".")[0]


In [12]:
tabela = tabela.head(10)

tickers = tabela.index

tickers

Index(['PSSA3', 'PLPL3', 'WIZC3', 'CMIN3', 'PETR4', 'PETR3', 'VLID3', 'VALE3',
       'LREN3', 'LEVE3'],
      dtype='object', name='Papel')

passo 2 - comfigurar o MT5

passo 3 - importar a biblioteca MT5

In [10]:
# ! pip install metatrader5



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import MetaTrader5 as mt5

passo 4 - inicializar o MT5

In [14]:
mt5.initialize()

True

passo 5 -configurar o dicionario de ordem e compra de ações

passo 6 

http://www.mql5.com/en/docs/python_metatrader5

tem que ir na documentação do meta trade e buscar o codigo

In [16]:
for ticker in tickers:

    print(ticker)

    info_acoes = mt5.symbol_info(ticker)
    mt5.symbol_select(ticker)
    tick_min = mt5.symbol_info(ticker).point
    preco = mt5.symbol_info_tick(ticker).ask
    quantidade = 100.0
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": quantidade,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco,
        "magic": 1,
        "comment": "Trades automáticos",
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result_compra = mt5.order_send(ordem_compra)

PSSA3


AttributeError: 'NoneType' object has no attribute 'point'